# Initial Setup

In [ ]:
# Inspect GPU# Inspect GPU
!nvidia-smi

Mon Apr 12 00:39:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# !unzip train_labels.npy.zip
# !unzip dev.npy.zip
# !unzip test.npy.zip
# !unzip train.npy.zip

Archive:  train_labels.npy.zip
  inflating: train_labels.npy        
Archive:  dev.npy.zip
  inflating: dev.npy                 
Archive:  test.npy.zip
  inflating: test.npy                
Archive:  train.npy.zip
  inflating: train.npy               


### Necessary extra libraries

In [ ]:
# Levenstein
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149806 sha256=564635d7805910ba23ebd17a6a60c2b550f42fd1af357a32b4f2875b3bd5499a
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
# CTC Decoder
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd -

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1063, done.
remote: Total 1063 (delta 0), reused 0 (delta 0), pack-reused 1063
Receiving objects: 100% (1063/1063), 759.71 KiB | 9.74 MiB/s, done.
Resolving deltas: 100% (513/513), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 13687, done.        
remote: Total 13687 (delta 0), reused 0 (delta 0), pack-reused 13687        
Receiving objects: 100% (13687/13687), 5.46 MiB | 20.11 MiB/s, done.
Resolving deltas: 100% (7880/7880), done.
Submodule path 'third_party/ThreadPool': checked out '9

In [ ]:
# Alternate edit distanec library
!git clone https://github.com/1ytic/pytorch-edit-distance
%cd pytorch-edit-distance
!python setup.py install
%cd -

Cloning into 'pytorch-edit-distance'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 27 (delta 11), reused 25 (delta 9), pack-reused 0
Unpacking objects: 100% (27/27), done.
/content/pytorch-edit-distance
     |████████████████████████████████| 194kB 9.0MB/s 
running install
running bdist_egg
running egg_info
creating torch_edit_distance.egg-info
writing torch_edit_distance.egg-info/PKG-INFO
writing dependency_links to torch_edit_distance.egg-info/dependency_links.txt
writing requirements to torch_edit_distance.egg-info/requires.txt
writing top-level names to torch_edit_distance.egg-info/top_level.txt
writing manifest file 'torch_edit_distance.egg-info/SOURCES.txt'
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  wa

# Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
import torch.optim as optim
from sklearn.metrics import accuracy_score
import time

import Levenshtein as lev
from ctcdecode import CTCBeamDecoder

In [ ]:
# Set up CUDA if available and set seeds fro reproducibility
cuda = torch.cuda.is_available()
torch.backends.cudnn.deterministic = True
torch.manual_seed(1)
if cuda: torch.cuda.manual_seed(1)
np.random.seed(1)

# Hyperparameters

In [ ]:
num_workers = 4 if cuda else 0
BATCH_SIZE = 64
N_EPOCHS = 10
DROPOUT_PROB = 0.4

# Data Import

In [ ]:
# # Import data
train_data = np.load('train.npy', allow_pickle=True)
train_labels = np.load('train_labels.npy', allow_pickle=True)
val_data = np.load('dev.npy', allow_pickle=True)
val_labels = np.load('dev_labels.npy', allow_pickle=True)

# dev = np.load('dev.npy',allow_pickle=True)
# dev_labels = np.load('dev_labels.npy', allow_pickle=True)
# test = np.load('test.npy', allow_pickle=True)
# train = np.load('train.npy', allow_pickle=True)
# train_labels = np.load('train_labels.npy',allow_pickle=True)
# print("dev.shape", dev.shape)
# print("dev_label.shape", dev_labels.shape)
# print("test.shape", test.shape)
# print("train.shape", train.shape)
# print("train_labels.shape", train_labels.shape)
from phoneme_list import *

Inspect shapes of the incoming data

In [ ]:
print("val_data.shape", val_data.shape)
print("val_label[0]", val_labels[0])
print("train_data[0].shape", train_data[0].shape)
print("train_labels[0]", train_labels[0])

val_data.shape (2200,)
val_label[0] [1, 23, 13, 33, 24, 20, 18, 20, 30, 38, 5, 33, 19, 33, 19, 40, 16, 14, 31, 33, 1]
train_data[0].shape (1184, 40)
train_labels[0] [1, 4, 25, 11, 4, 40, 33, 36, 18, 19, 40, 25, 3, 23, 19, 21, 5, 37, 12, 5, 31, 33, 15, 33, 19, 25, 38, 19, 10, 12, 5, 29, 30, 13, 24, 5, 31, 5, 40, 18, 4, 11, 9, 19, 25, 16, 7, 25, 11, 1, 4, 25, 11, 18, 19, 40, 18, 4, 37, 19, 26, 31, 27, 16, 5, 24, 19, 23, 39, 14, 23, 20, 30, 20, 23, 8, 11, 5, 29, 3, 25, 12, 5, 16, 4, 22, 33, 5, 37, 25, 27, 30, 3, 9, 14, 20, 18, 4, 37, 19, 26, 33, 15, 22, 5, 25, 29, 23, 15, 31, 1, 4, 40, 5, 25, 3, 30, 17, 39, 5, 24, 5, 25, 33, 6, 25, 18, 19, 40, 27, 25, 9, 19, 18, 4, 16, 1]


# Dataset & Data Loader Definition for Training

In [ ]:
class RNNDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
 
    def __getitem__(self, index):
        x = self.X[index]
        if self.y is not None:
          y = self.y[index]
          return (torch.Tensor(x), torch.LongTensor(y))
        else:
          return torch.Tensor(x)

In [ ]:
def collate(batch):
  Xs, ys, x_lengths, y_lengths = [], [], [], []

  for x,y in batch:
      Xs.append(x)
      ys.append(y)
      x_lengths.append(x.shape[0])
      y_lengths.append(len(y))
  return (pad_sequence(Xs, batch_first=False), x_lengths), (pad_sequence(ys, batch_first=True), y_lengths)


In [ ]:
def collate_test(batch):
  Xs, x_lengths = [], []

  for x in batch:
    Xs.append(x)
    x_lengths.append(x.shape[0])

  return (pad_sequence(Xs, batch_first=False), x_lengths)

In [ ]:
batch_size = 64

train_dataset = RNNDataset(train_data, train_labels)

train_loader = DataLoader(train_dataset, batch_size= batch_size  , shuffle=True,num_workers=num_workers, pin_memory=True,collate_fn =collate)

In [ ]:
val_dataset = RNNDataset(val_data, val_labels) 
val_loader = DataLoader(val_dataset, batch_size= batch_size  , shuffle=False,num_workers=num_workers, pin_memory=True,collate_fn =collate)

# Model Definition

In [ ]:

# Model Definition
class BiLSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
      super(BiLSTMModel, self).__init__()
      
      self.lstm = nn.LSTM(input_size, hidden_size, 
                          num_layers, bidirectional=True, 
                          batch_first=True, dropout=DROPOUT_PROB)
      # self.relu = nn.ReLU()
      self.output = nn.Linear(hidden_size*2, output_size) # hidden size*2 because it's a bi-LSTM

  def forward(self, x, seq_lengths):
    x = pack_padded_sequence(x.float(), seq_lengths, batch_first=False, enforce_sorted=False)
    x = self.lstm(x)[0]
    x, out_lengths = pad_packed_sequence(x, batch_first=False)
    x = self.output(x).log_softmax(2)# expected by CTC
    return x, out_lengths

In [ ]:
# Build the model
num_phonemes = 42
num_layers = 4
hidden_size = 256

# model = BiLSTMModel(40, hidden_size, num_phonemes, num_layers)

device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

learningRate = 2e-3
criterion = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=learningRate, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8, verbose=True)

BiLSTMModel(
  (lstm): LSTM(40, 256, num_layers=4, batch_first=True, dropout=0.4, bidirectional=True)
  (output): Linear(in_features=512, out_features=42, bias=True)
)
Adjusting learning rate of group 0 to 2.0000e-03.


### CTC Decoder

In [ ]:
decoder = CTCBeamDecoder(
    PHONEME_LIST,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=100,
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)

In [ ]:
def get_label_string(i):
  return PHONEME_MAP[i.numpy().astype(int)]

In [ ]:
def get_avg_edit_distance(predictions, target):
  distances = []
  for p, t in zip(predictions, target):
    distances.append(lev.distance(p, t))
  return np.mean(distances)

In [ ]:
def decode_sequence(prediction):
  output_string = []
  lengths = []
  for p in prediction:
    # print(p)
    decoded_string = list(map(get_label_string, list(p)))

    output_string.append((''.join(decoded_string)))
  return output_string

# Training & Validation

In [ ]:
# Training function using mixed precision
# NEW
scaler = torch.cuda.amp.GradScaler()

def train_epoch(model, train_loader, criterion, optimizer, epoch_num):
    print(f"Epoch {epoch_num}...")
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        
        data_x = data[0].to(device)
        data_length = torch.Tensor(data[1])

        target_y = target[0].to(device)
        target_length = torch.LongTensor(target[1]).to(device)

        outputs, out_length = model(data_x, data_length)

        loss = criterion(outputs, target_y, out_length, target_length)
        running_loss += loss.item()

        loss.backward() # Calculate gradients

        optimizer.step() # Apply gradient descent step

    end_time = time.time()
    
    running_loss /= len(train_loader)
    # acc = (correct_predictions/total_predictions)*100.0
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    # print('Training Accuracy: ', acc, '%')
    print('Train time (min):', (end_time - start_time)/60)
    print()
    return running_loss#, acc

In [ ]:
# Validation/ Evaluation Function
def val_model(model, test_loader, criterion):
    print('Validating...')
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        prediction_distances = []

        start_time = time.time()
        for batch_idx, (data, target) in enumerate(val_loader):   
            
            data_x = data[0].to(device)
            data_length = torch.Tensor(data[1])

            target_y = target[0].to(device)
            target_length = torch.LongTensor(target[1]).to(device)

            outputs, out_lengths = model(data_x, data_length)

            loss = criterion(outputs, target_y, out_lengths, target_length)
            running_loss += loss.item()

            # predicted = torch.transpose(outputs, 0, 1)# match to CTC Beam decoder

            # align = []

            # target_y_list = []
            
            # predict_beam_results, _, _, predict_out_lens = decoder.decode(
            #     predicted.data.cpu(),torch.IntTensor(out_lengths.to((torch.int))))
            
            # for i, out_len in enumerate(predict_out_lens):
            #     align.append(predict_beam_results[i, 0, :predict_out_lens[i, 0]])
            #     target_y_list.append(target_y[i, :target_length[i]].cpu())

            # # print("ALIGN")
            # pred_seq = decode_sequence(align)
            # # print("Align:", align)
            # # print("Predicted:",pred_seq)
            # # print("TARGET")
            # target_seq = decode_sequence(target_y_list)
            # # print("target_y:", target_y)
            # # print("Target:", target_seq)
            # # print("===="*20)
          
            # dist = get_avg_edit_distance(pred_seq,target_seq)
            # # print(dist)
            
            # prediction_distances.append(dist)
            # total_predictions += target.size(0)
            # correct_predictions += (predicted == target).sum().item()

        end_time = time.time()
        running_loss /= len(test_loader)
        # distance_loss = np.mean(prediction_distances)
        # acc = (correct_predictions/total_predictions)*100.0
        print('Validation Loss: ', running_loss)
        # print('Validation Distance Loss: ', distance_loss)
        print('Validation time (min):', (end_time - start_time)/60)
        print()
        return running_loss


In [ ]:
# model.load_state_dict(torch.load(save_location_raw+"Trial1-epoch32.pth"))

<All keys matched successfully>

In [ ]:
# Run the model
train_losses = []
# train_accuracies = []
train_distances = []
test_losses = []
# test_accuracies = []
trialNum = 1
best_epoch = 0
winner = ""

current_best = 999999999999999 # start arbitrarily high

for epoch in range(44, 60):    
    train_loss = train_epoch(model, train_loader, criterion, optimizer, epoch)
    test_loss = val_model(model, val_loader, criterion)
    scheduler.step()
    
    train_losses.append(train_loss)
    # train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    # train_distances.append(test_dist)

    if test_loss < current_best:
      # Get old model to be deleted
      old_model = f"Trial{trialNum}-epoch{best_epoch}.pth"

      # Save new best model
      current_best = test_loss
      best_epoch = epoch
      winner = f"Trial{trialNum}-epoch{epoch}"
      filename = winner+".pth"
      torch.save(model.state_dict(), filename)

      # Delete old model if exists
      if os.path.exists(old_model):
        os.remove(old_model)
    print('='*20)

Epoch 44...
Training Loss:  0.29931668616962015 Time:  361.84723353385925 s
Train time (min): 6.030787225564321

Validating...
Validation Loss:  0.5555062336581094
Validation time (min): 0.29434130191802976

Adjusting learning rate of group 0 to 1.6000e-03.
Epoch 45...
Training Loss:  0.276749989508014 Time:  361.72945618629456 s
Train time (min): 6.028824269771576

Validating...
Validation Loss:  0.51199922646795
Validation time (min): 0.29397430419921877

Adjusting learning rate of group 0 to 1.2800e-03.
Epoch 46...
Training Loss:  0.24880854059991084 Time:  362.02290868759155 s
Train time (min): 6.0337151447931925

Validating...
Validation Loss:  0.5042358713490622
Validation time (min): 0.2936131040255229

Adjusting learning rate of group 0 to 1.0240e-03.
Epoch 47...
Training Loss:  0.22158350070056163 Time:  362.1181709766388 s
Train time (min): 6.0353028496106464

Validating...
Validation Loss:  0.4937239127499717
Validation time (min): 0.293220591545105

Adjusting learning rate 

KeyboardInterrupt: ignored

# Load Best Model for Testing

In [ ]:
winning_model = model
winning_model.load_state_dict(torch.load(winner+".pth"))
winning_model.eval()

BiLSTMModel(
  (lstm): LSTM(40, 256, num_layers=4, batch_first=True, dropout=0.4, bidirectional=True)
  (output): Linear(in_features=512, out_features=42, bias=True)
)

In [ ]:
# test_loss, test_dist = val_model(winning_model, val_loader, criterion)

# Data Loader Definition for Testing

In [ ]:
test_data = np.load('test.npy', allow_pickle=True)
test_dataset = RNNDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size= batch_size  , shuffle=False,num_workers=num_workers, pin_memory=True,collate_fn =collate_test)

# Test Final Model on Testing Data

In [ ]:
def test_on_final_model(model, test_loader, criterion):
  print('Final Testing...')
  with torch.no_grad():
    model.eval()

    p = []
    i = 0
    for batch_idx, (data) in enumerate(test_loader):
      # print(data[0])

      data_x = data[0].to(device)
      data_length = torch.Tensor(data[1])

      outputs, out_lengths = model(data_x, data_length)

      predicted = torch.transpose(outputs, 0, 1)# match to CTC Beam decoder

      align = []
      predict_beam_results, _, _, predict_out_lens = decoder.decode(
                predicted.data.cpu(),torch.IntTensor(out_lengths.to((torch.int))))
      
      for i, out_len in enumerate(predict_out_lens):
          align.append(predict_beam_results[i, 0, :predict_out_lens[i, 0]])
      
      p.append(decode_sequence(align))

    p = np.concatenate(p)
    return p # This will be written to csv

In [ ]:
p = test_on_final_model(winning_model, test_loader, criterion)
print('Done')
print(p)

Final Testing...
Done
['.prlhkoRdiNlIRAnthDhbOWhndOAtDhfaRDrhndhvDhHol.hndlukthloNDhvisthvhgaRdhnWok.kaphtid.WiTklOslISEvingRAs.AndboRdrdWiTshmRUdhnd.imitoRhtempt.htRhvrI.'
 '.DhpwdhpvEprzkhntenstinhWotr.AndDIeR.putinhvylhAkShntUshplyDhvAkim.leftbyDhkandinsEShn.hvDhmist.RwzizitselfintUhWrldWen.'
 '.AndfiRsDwbikhzymfAniS.inimsImOmoR.' ...
 '.DhmhdhnaghnyzEShnhv.indistRIWrksinDhsEmdrekShnolsObynhvrlyn.'
 '.liftiNDIyrnHAmrDhtHhNDhtDhpoRthl.HAstrpRindIvhshmhnzWicWhzAnsrdbyWhnhvDhgabrnrzbansrvhns.hfRIboRniNgliSmAn.bhtnwhsevhn?uRslEz.'
 '.sOyWOlgibwtDislo.DAtmymenShlnevrlIv?UhlOn.']


In [ ]:
len(p)